## 3. Working with Generics
Generics allow you to define the specification of the data type of programming elements in a class or a method, until it is actually used in the program. In other words, generics allow you to write a class or method that can work with any data type.

In [4]:
public class MyGenericArray<T> {
    private T[] array;
      
    public MyGenericArray(int size) {
        array = new T[size + 1];
    }

    public T getItem(int index) {
        return array[index];
    }
    
    public void setItem(int index, T value) {
        array[index] = value;
    }
}


class Tester {
    public static void Main() {
        int len = 5;

        //declaring an int array
        MyGenericArray<int> intArray = new MyGenericArray<int>(len);
         
        //setting values
        for (int c = 0; c < len; c++) {
            intArray.setItem(c, c*5);
        }
         
        //retrieving the values
        for (int c = 0; c < len; c++) {
            Console.Write(intArray.getItem(c) + " ");
        }
        Console.WriteLine();
         
        //declaring a character array
        MyGenericArray<char> charArray = new MyGenericArray<char>(len);
         
        //setting values
        for (int c = 0; c < 5; c++) {
            charArray.setItem(c, (char)(c+97));
        }
         
        //retrieving the values
        for (int c = 0; c< 5; c++) {
            Console.Write(charArray.getItem(c) + " ");
        }
        Console.WriteLine();
    }
}


Tester.Main();

0 5 10 15 20 
a b c d e 


Some articles and papers might lead you to believe that generics are useful only in the context of collections. That’s not true. There are many other ways to use generics. You can use them to create interfaces, event handlers, common algorithms, and more.

### Item 18: Always Define Constraints That Are Minimal and Sufficient

### Item 19: Specialize Generic Algorithms Using Runtime Type Checking

### Item 20: Implement Ordering Relations with IComparable<T> and IComparer<T>


In [10]:
public struct Customer : IComparable<Customer>, IComparable {
    public readonly string name;
    
    public Customer(string name) {
        this.name = name;
    }
    
    // IComparable<Customer> Members
    public int CompareTo(Customer other) => name.CompareTo(other.name);
    
    // IComparable Members
    int IComparable.CompareTo(object obj) {
        if (!(obj is Customer)) {
            throw new ArgumentException("Argument is not a Customer", "obj");
        }
        Customer otherCustomer = (Customer)obj;
        return this.CompareTo(otherCustomer);
    }
}


In [12]:
List<Customer> customers = new List<Customer>(); 
customers.Add(new Customer("Simon"));
customers.Add(new Customer("Jake"));
customers.Add(new Customer("James"));

customers.Sort();
foreach(var customer in customers) {
    Console.WriteLine(customer.name);
}

Jake
James
Simon


There’s just too much to dislike about the classic version of IComparable. You’ve got to check the runtime type of the argument. Incorrect code could legally call this method with anything as the argument to the CompareTo method. More so, proper arguments must be boxed and unboxed to provide the actual comparison. That’s an extra runtime expense for each compare.

Adding operators to this code...

In [13]:
public struct Customer : IComparable<Customer>, IComparable {
    public readonly string name;
    
    public Customer(string name) {
        this.name = name;
    }
    
    // IComparable<Customer> Members
    public int CompareTo(Customer other) => name.CompareTo(other.name);
    
    // IComparable Members
    int IComparable.CompareTo(object obj) {
        if (!(obj is Customer)) {
            throw new ArgumentException("Argument is not a Customer", "obj");
        }
        Customer otherCustomer = (Customer)obj;
        return this.CompareTo(otherCustomer);
    }
    
    public static bool operator < (Customer left, Customer right) => left.CompareTo(right) < 0;
    
    public static bool operator <= (Customer left, Customer right) => left.CompareTo(right) <= 0;
    
    public static bool operator > (Customer left, Customer right) => left.CompareTo(right) > 0;
    
    public static bool operator >= (Customer left, Customer right) => left.CompareTo(right) >= 0;
}

In [14]:
var simon = new Customer("Simon");
var james = new Customer("James");

In [18]:
simon > james

True

In [19]:
simon >= james

True

In [16]:
simon < james

False

In [20]:
simon <= james

False

### Item 21: Always Create Generic Classes That Support Disposable Type Parameters

In [21]:
public interface IEngine {
    void DoWork();
}

public class EngineDriverOne<T> where T: IEngine, new() {
    public void GetThingsDone() {
        T driver = new T();
        driver.DoWork();
    }
}

You may have introduced a resource leak if T implements IDisposable. In every case where you create a local variable of type T, you need to check whether T implements IDisposable and, if so, dispose of it correctly.

In [22]:
public class EngineDriverOne<T> where T: IEngine, new() {
    public void GetThingsDone() {
        T driver = new T();
        using (driver as IDisposable) {
            driver.DoWork();
        }
    }
}

If T does not implement IDisposable, then the value of this local variable is null. In those cases, the compiler does not call Dispose(), because it checks against null before doing this extra work.

### Item 22: Support Generic Covariance and Contravariance